# TASK 1

## Task: Update a factory automation component

Welcome to your first task as the factory automation operator!

We currently have a Cisco Kinetic EFM instance running in our factory edge. It's already pulling data from our 3D printer for monitoring and control perposes.

<Image of Cisco EFM Designer>

We have a new version that provides new features and needed functionality for the business, in our case, a brand new monitoring dashboard for the factory floor team.

Lets upgrade our component using the Kubernetes API!

## How to play

Some input below will be needed, fields may be missing, read the comments to work out what needs completing to sucessfully remediate the issue!

To run a block of code below, hold SHIFT+ENTER. 
NO NEW LINES OF CODE ARE NEEDED, WE'RE NOT THAT MEAN!


In [8]:
# First things first! We need to ask CCP for access credentials to our new cluster.
# Theres an API for this, as long as you have permission!
%env CRED_API_IP=10.10.20.110

# Here, we'll need your details from green box on the challenge homepage!
%env CCP_API_USER=factory-RDRLW
%env CCP_API_PASSWORD=factory-RDRLW 
%env CCP_API_IP=10.10.20.209

# Shift + Enter to run this section, we're just setting those variables for later!

env: CRED_API_IP=10.10.20.110
env: CCP_API_USER=factory-RDRLW
env: CCP_API_PASSWORD=factory-RDRLW
env: CCP_API_IP=10.10.20.209


In [9]:
%%bash
mkdir ./temp ~/.kube/

# Authenticating to the Kubernetes API.
# Finding cluster ID using NAME.
# Pulling KubeConfig and Kubernetes API endpoint details.

# Auth to CCP API as new user, get auth token as cookie."
curl -k -c ./temp/cookie.txt -H "Content-Type:application/x-www-form-urlencoded" -d "username=$CCP_API_USER&password=$CCP_API_PASSWORD" https://${CRED_API_IP}/2/system/login/

echo "Get the cluster UUID for ${CCP_API_USER} :"
cluster_uuid=`curl -k -b ./temp/cookie.txt https://${CRED_API_IP}/2/clusters| jq -r '.[]|.name, .uuid' | grep -A1 ${CCP_API_USER} | sed -n 2p`
echo ${cluster_uuid} 
echo ${cluster_uuid} > .clusteruuid

# Download our clusters kubeconfig file using UUID
curl -k -X GET -b ./temp/cookie.txt https://${CRED_API_IP}/2/clusters/${cluster_uuid}/env > ~/.kube/config

Get the cluster UUID for factory-RDRLW :
ce52952e-721e-460e-8371-7778abc2290b


mkdir: cannot create directory ‘./temp’: File exists
mkdir: cannot create directory ‘/home/automationchallenge/.kube/’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    45    0     0  100    45      0    169 --:--:-- --:--:-- --:--:--   170
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8796    0  8796    0     0  12016      0 --:--:-- --:--:-- --:--:-- 12016
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6341    0  6341    0     0   5107      0 --:--:--  0:00:01 --:--:--  5109


### Now we have credentials for our Kubernetes cluster from CCP, we can use the kubernetes API's and CLI's

In [10]:
# See the nodes that make up our factory kubernetes cluster using the Kubernetes CLI
! kubectl get nodes

# See the current deplo|yment in our edge kubernetes cluster using the Kubernets CLI
! kubectl get deployments



NAME                             STATUS   ROLES    AGE   VERSION
factory-rdrlw-masterbc22702878   Ready    master   4h    v1.11.5
factory-rdrlw-worker651db66be2   Ready    <none>   4h    v1.11.5
No resources found.


## Kubernetes Deployments 

Deployments tell kubernetes to run one or more copies of the same Docker container image, known as replicas, they also provide logic to perform "rolling upgrades" on deployments with many replicas, allowing to move to a new version of a given containerized application without downtime for the users.

We can see the indevidual pods (container instances) powering a given deployment by running `kubectl describe deployment <deployment name>`

The deployment we care about is `efm-kinetic-printer1`....


In [ ]:
# Try inspecting the deployment we want to update (you'll need the name...)
! kubectl describe deployment ...


## Kubernetes Services

You may have noticed in the output above, each pod had an internal IP within the kubernetes cluster, this is no good for instances of a single app all running concurrently for resilience and handling load (which is what a `Deployment` construct in kubernetes is designed for!)

So we'll need something to put a public/external IP address on the pool of pods and load balance access between them. In kubernetes, these are known as services.

In [ ]:
# List the kubernetes service objects in our factory cluster.
! kubectl get services

We have a few services already, notice our `efm-kinetic-printer1-service` 
(these names are arbitrary and upto the person automating the deployments and services against the cluster).

We can see we have an external IP, we could select just that service as below..

In [ ]:
# show just one service
! kubectl get service efm-kinetic-printer1-service

# We can also ask for kubectl output in JSON, allowing us to filter the bits we want easily!
! kubectl get service efm-kinetic-printer1-service -o json | jq -r '.spec.clusterIP'

# Now we know howto get the IP, we can easily give you the EFM UI URL for your environment...
! echo "https://`kubectl get service efm-kinetic-printer1-service -o json | jq -r .spec.clusterIP`/dataflow.html"

## Access the EFM via it's service IP.

Click the URL above to open the EFM UI to see the Cisco Kinetic UI!

Log in with User: `EFMAdmin` and Password: `cisco123`.

![EFM Login](images/efm-login.png)

Drill down into `Downstream > Dataflow` on the left hand side, and press the button next to the `getJobs` item to load the workflow.

![Expand Data Collection Items](images/efm-expand.png)

Here you will see the data collector polling our 3D Printer for information about its current jobs, you will see the responses coming back if you inspect the `strLoader` object in the diagram.

![strLoader object in flow](images/efm-flow.png)